In [3]:
%load_ext autoreload
%autoreload 2
import librosa
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os

import audioUtils

#import DyTrainingDataGenerator
import SpeechModels
%matplotlib inline  

In [4]:
#Convert WAV to Numpy Matrix
def convertWAV2Numpy():
    print('Converting test set WAVs to numpy files')
    audioUtils.WAV2Numpy(basePath + '/test/')
    print('Converting training set WAVs to numpy files')
    audioUtils.WAV2Numpy(basePath + '/train/')

In [5]:
# prepare the labels for training
basePath = "sd_GSCmdV2"

def preparegooglespeechcmd():
    
    GSCmdV2Categs = {'unknown' : 0, 'silence' : 0, '_unknown_' : 0, '_silence_' : 0, '_background_noise_' : 0, 'yes' : 2, 
                         'no' : 3, 'up' : 4, 'down' : 5, 'left' : 6, 'right' : 7, 'on' : 8, 'off' : 9, 'stop' : 10, 'go' : 11,
                         'zero' : 12, 'one' : 13, 'two' : 14, 'three' : 15, 'four' : 16, 'five' : 17, 'six' : 18, 
                         'seven' : 19,  'eight' : 20, 'nine' : 1, 'backward':21, 'bed':22, 'bird':23, 'cat':24, 'dog':25,
                         'follow':26, 'forward':27, 'happy':28, 'house':29, 'learn':30, 'marvin':31, 'sheila':32, 'tree':33,
                         'visual':34, 'wow':35}
    numGSCmdV2Categs = 36
    
    
     #read split from files and all files in folders
    testWAVs = pd.read_csv(basePath+'/train/testing_list.txt', sep=" ", header=None)[0].tolist()
    valWAVs  = pd.read_csv(basePath+'/train/validation_list.txt', sep=" ", header=None)[0].tolist()
    

    testWAVs = [os.path.join(basePath+'/train/', f + '.npy') for f in testWAVs if f.endswith('.wav')]
    valWAVs  = [os.path.join(basePath+'/train/', f + '.npy') for f in valWAVs if f.endswith('.wav')]
    allWAVs  = []
    for root, dirs, files in os.walk(basePath+'/train/'):
        allWAVs += [root+'/'+ f for f in files if f.endswith('.wav.npy')]
    trainWAVs = list( set(allWAVs)-set(valWAVs)-set(testWAVs) )
    
    #get categories
    trainWAVlabels    = [_getFileCategory(f, GSCmdV2Categs) for f in trainWAVs]
    #print(trainWAVlabels)
    #build dictionaries
    trainWAVlabelsDict    = dict(zip(trainWAVs, trainWAVlabels))
    #print(trainWAVlabelsDict)
    #info dictionary
    trainInfo = {'files' : trainWAVs, 'labels' : trainWAVlabelsDict}
    gscInfo = {'train' : trainInfo}
    
    print('Done preparing Google Speech commands dataset version ')
    
    return gscInfo, numGSCmdV2Categs

In [6]:
def _getFileCategory(file, catDict):
    """
    Receives a file with name sd_GSCmdV2/train/<cat>/<filename> and returns an integer that is catDict[cat]
    """
    categ = os.path.basename(os.path.dirname(file))
    print(categ)
    return catDict.get(categ,0)

In [7]:
numberDirsDict = {1:'one',2:'two',3:'three',4:'four',5:'five',6:'six',7:'seven',8:'eight',9:'nine'}
def _getDyFileCategory(file, fileDict):
    categ = os.path.basename(os.path.dirname(file))
    foldername = None
    if len(categ) > 2 :
        foldername = categ[2:]
    else:
        foldername = numberDirsDict.get(int(categ))
        
    return fileDict.get(foldername)
        

In [8]:
def preparedyspeechcmd():
    DYSCmdCategs = {'unknown' : 0, 'one': 1, 'two' : 2, 'three' : 3, 'four' : 4, 'five' : 5, 'six' : 6, 'seven' : 7, 'eight' : 8, 'nine' : 9, 'close' : 10, 'up' : 11,
                    'down' : 12, 'previous' : 13, 'next' : 14, 'in' : 15, 'out' : 16,'left' : 17, 'right' : 18, 'home' : 19}
    numDYCmdCategs = 20
    _baseDir = 'linzy_command'
    firstLevelDirs = getNextLevelDirs(_baseDir)
    trainfiles = list()
    for folder in firstLevelDirs:
        d = os.path.join(_baseDir,folder)
        #print(d)
        files = [os.path.join(d,f+'.npy') for f in next(os.walk(d))[2] if f.endswith('.wav')]
        #print(files)
        #trainfiles.append([os.path.join(d,f+'.npy') for f in next(os.walk(d))[2] if f.endswith('.wav')])
        trainfiles += files
        
    #print(trainfiles)
    trainFilesLabels    = [_getDyFileCategory(f, DYSCmdCategs) for f in trainfiles]
    trainWAVlabelsDict    = dict(zip(trainfiles, trainFilesLabels))
    print(trainWAVlabelsDict)
    
    trainInfo = {'files': trainfiles, 'labels' : trainWAVlabelsDict}
    gscInfo = {'train' : trainInfo}
    
    return gscInfo, numDYCmdCategs
    

In [9]:
def getNextLevelDirs(root):
     return next(os.walk(root))[1]

In [10]:
sr=16000 #we know this one for google audios
iLen = 16000
batchSize = 64
#preparegooglespeechcmd()
_dscInfo, _numofCategs = preparedyspeechcmd()

#print out the _gsc info
#print(_dscInfo.keys())
#print(_dscInfo['train'].keys())
#print(len(_dscInfo['train']['files']))
#print(_numofCategs)
#print(_gsc)

{'linzy_command/14next/LinZY03_14_4.wav.npy': 14, 'linzy_command/14next/LinZY03_14_5.wav.npy': 14, 'linzy_command/14next/LinZY03_14_6.wav.npy': 14, 'linzy_command/14next/LinZY03_14_2.wav.npy': 14, 'linzy_command/14next/LinZY03_14_3.wav.npy': 14, 'linzy_command/14next/LinZY03_14_1.wav.npy': 14, 'linzy_command/18right/LinZY03_18_1.wav.npy': 18, 'linzy_command/18right/LinZY03_18_3.wav.npy': 18, 'linzy_command/18right/LinZY03_18_2.wav.npy': 18, 'linzy_command/18right/LinZY03_18_6.wav.npy': 18, 'linzy_command/18right/LinZY03_18_5.wav.npy': 18, 'linzy_command/18right/LinZY03_18_4.wav.npy': 18, 'linzy_command/11up/LinZY03_11_3.wav.npy': 11, 'linzy_command/11up/LinZY03_11_2.wav.npy': 11, 'linzy_command/11up/LinZY03_11_1.wav.npy': 11, 'linzy_command/11up/LinZY03_11_5.wav.npy': 11, 'linzy_command/11up/LinZY03_11_4.wav.npy': 11, 'linzy_command/11up/LinZY03_11_6.wav.npy': 11, 'linzy_command/9/LinZY03_09_1.wav.npy': 9, 'linzy_command/9/LinZY03_09_2.wav.npy': 9, 'linzy_command/9/LinZY03_09_3.wav.npy

In [11]:

def all_training_data_generation(list_of_files, _labels):
        
        X = np.empty((len(list_of_files),iLen)) #64 files, each file is 16000 long
        y = np.empty((len(list_of_files)),dtype=int)
        
        #Start to generate the training data
        for i, _f in enumerate(list_of_files):
            #print("current _f is : ",_f)
            #load npy file
            curX = np.load(_f)
            
            #check equal,smaller, or bigger
            #and truncate or padding
            #curX could be bigger or smaller than self.dim
            if curX.shape[0] == iLen:
                X[i] = curX
                #print('Same dim')
            elif curX.shape[0] > iLen: #bigger
                #we can choose any position in curX-self.dim
                randPos = np.random.randint(curX.shape[0]-iLen) 
                X[i] = curX[randPos:randPos+iLen]
                #print('File dim bigger')
            else: #smaller
                randPos = np.random.randint(iLen-curX.shape[0])
                X[i,randPos:randPos+curX.shape[0]] = curX
                #print('File dim smaller')
        # Store class
            y[i] = _labels[_f]

        return X, y

In [12]:
# generating all training data
x_train, y_train = all_training_data_generation(_dscInfo['train']['files'],_dscInfo['train']['labels'])
for i in range(len(x_train)) :
    print("x_train : {}, y_train : {}".format(x_train[i],y_train[i]))

x_train : [-0.00021362 -0.00036621 -0.00042725 ...  0.00082397  0.00045776
  0.00067139], y_train : 14
x_train : [ 0.00186157  0.00210571  0.00186157 ...  0.00027466 -0.00021362
 -0.00018311], y_train : 14
x_train : [ 0.          0.          0.         ... -0.00253296 -0.00183105
 -0.0015564 ], y_train : 14
x_train : [-0.00100708 -0.00143433 -0.00152588 ... -0.0012207  -0.00085449
 -0.00048828], y_train : 14
x_train : [ 0.00015259  0.00018311  0.00018311 ... -0.00109863  0.00064087
  0.00097656], y_train : 14
x_train : [-2.65289307e-01 -2.64465332e-01 -2.70629883e-01 ... -1.83105469e-04
 -4.88281250e-04 -3.35693359e-04], y_train : 14
x_train : [ 0.00177002  0.00213623  0.00234985 ... -0.00106812 -0.00186157
 -0.00424194], y_train : 18
x_train : [0.01019287 0.01113892 0.01113892 ... 0.00683594 0.00656128 0.00762939], y_train : 18
x_train : [-0.00204468 -0.00344849 -0.00341797 ...  0.00030518  0.00030518
  0.00039673], y_train : 18
x_train : [ 0.00671387  0.00979614  0.01071167 ... -0.00

In [13]:
#trainingData = DyTrainingDataGenerator.DySpeechGen(_dscInfo['train']['files'],_dscInfo['train']['labels'])
def testDataenerator(files,lbls):
    #_dsc['train']['files'],_dsc['train']['labels']
    print("_dscInfo'train']['files'] : ",_dscInfo['train']['files'])
    trainingData = DyTrainingDataGenerator.DySpeechGen(files,lbls)
    _audios, _classes = trainingData.__getitem__(0)
    print(trainingData.__len__())
    return _audios, _classes

In [14]:
#audios, classes = testDataenerator(_dscInfo['train']['files'],_dscInfo['train']['labels'])
#classes

In [15]:
#librosa.output.write_wav('file.wav', audios[5], sr, norm=False)
#plt.plot(audios[5])

In [16]:
#Start to Build the mfcc layer
'''
from keras.models import Sequential
from kapre.time_frequency import Melspectrogram, Spectrogram
from kapre.utils import Normalization2D

melspecModel = Sequential()

melspecModel.add(Melspectrogram(n_dft=1024, n_hop=128, input_shape=(1, iLen),
                         padding='same', sr=sr, n_mels=80,
                         fmin=40.0, fmax=sr/2, power_melgram=1.0,
                         return_decibel_melgram=True, trainable_fb=False,
                         trainable_kernel=False,
                         name='mel_stft') )

melspecModel.add(Normalization2D(int_axis=0))

melspecModel.summary()
'''

"\nfrom keras.models import Sequential\nfrom kapre.time_frequency import Melspectrogram, Spectrogram\nfrom kapre.utils import Normalization2D\n\nmelspecModel = Sequential()\n\nmelspecModel.add(Melspectrogram(n_dft=1024, n_hop=128, input_shape=(1, iLen),\n                         padding='same', sr=sr, n_mels=80,\n                         fmin=40.0, fmax=sr/2, power_melgram=1.0,\n                         return_decibel_melgram=True, trainable_fb=False,\n                         trainable_kernel=False,\n                         name='mel_stft') )\n\nmelspecModel.add(Normalization2D(int_axis=0))\n\nmelspecModel.summary()\n"

In [17]:
#melspec = melspecModel.predict( audios.reshape((-1,1,iLen)) )
#melspec.shape

# Models

Create Keras models to see if the generators are working properly

In [18]:
from keras.models import Model, load_model

from keras.layers import Input, Activation, Concatenate, Permute, Reshape, Flatten, Lambda, Dot, Softmax
from keras.layers import Add, Dropout, BatchNormalization, Conv2D, Reshape, MaxPooling2D, Dense#, LSTM, Bidirectional
from keras import backend as K
from keras.utils import to_categorical
#from keras.callbacks import ModelCheckpoint
#from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
#from keras import backend as K
from keras import optimizers
from keras.optimizers import SGD
from kapre.time_frequency import Melspectrogram, Spectrogram
from keras_tqdm import TQDMNotebookCallback

In [23]:
#self-attention LSTM
#model = SpeechModels.AttRNNSpeechModel(_numofCategs, samplingrate = sr, inputLength = iLen)
model = SpeechModels.ConvSpeechModel(_numofCategs, samplingrate = sr, inputLength = iLen)

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(optimizer='sgd', loss = ['sparse_categorical_crossentropy'], metrics=['sparse_categorical_accuracy']) 
model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy'], metrics=['sparse_categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 16000)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 1, 16000)          0         
_________________________________________________________________
mel_stft (Melspectrogram)    (None, 80, 125, 1)        1091664   
_________________________________________________________________
normalization2d_2 (Normaliza (None, 80, 125, 1)        0         
_________________________________________________________________
permute_2 (Permute)          (None, 125, 80, 1)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 80, 20)       120       
_________________________________________________________________
batch_normalization_2 (Batch (None, 125, 80, 20)       80        
__________

In [27]:
import math
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.4
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
            math.floor((1+epoch)/epochs_drop))
    
    if (lrate < 4e-5):
        lrate = 4e-5
      
    print('Changing learning rate to {}'.format(lrate))
    return lrate
#lrate = LearningRateScheduler(step_decay)

In [28]:
#earlystopper = EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10, verbose=1)
#checkpointer = ModelCheckpoint('model-attRNN.h5', monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True)
#checkpointer = ModelCheckpoint('model-Conv.h5', monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True)

#results = model.fit_generator(trainingData, validation_data = None, epochs = 40, use_multiprocessing=False, workers=1,
                    #callbacks=[earlystopper, checkpointer, lrate])
#result = model.fit(x_train,y_train, epochs = 20, batch_size = 64)

In [30]:
#callbacks_list = [checkpointer]
result = model.fit(x_train,y_train, epochs = 20, batch_size = 64, callbacks=[TQDMNotebookCallback])

ValueError: ('Some keys in session_kwargs are not supported at this time: %s', dict_keys(['callbacks']))